# Ingest & Scrape Notebook

Use this notebook to scrape raw data, chunk it, and upload to the API for QA generation.


In [ ]:
API_BASE = 'http://localhost:8000'
EMAIL = 'provider@example.com'
PASSWORD = 'password'


In [ ]:
import httpx, json, pathlib
from pathlib import Path


def login(email, password):
    with httpx.Client(base_url=API_BASE) as c:
        r = c.post('/auth/login', data={'username': email, 'password': password})
        r.raise_for_status()
        return r.json()['access_token']


token = login(EMAIL, PASSWORD)
token[:10]


In [ ]:
# Prepare a small chunks.jsonl from dataset/chunks/chunks.jsonl
src = Path('../../dataset/chunks/chunks.jsonl')
dst = Path('sample_chunks.jsonl')
data = []
for i, line in enumerate(src.read_text().splitlines()):
    if i >= 5: break
    data.append(line)
dst.write_text('\n'.join(data))
dst, len(data)


In [ ]:
# Upload to API for QA generation
with httpx.Client(base_url=API_BASE, headers={'Authorization': f'Bearer {token}'}) as c:
    files = {'f': (dst.name, dst.read_bytes(), 'application/json')}
    r = c.post('/upload/file', files=files)
    r.status_code, r.text
